In [7]:
# #https://archive.materialscloud.org/record/2019.0020/v1
# import pandas as pd
# from jarvis.db.jsonutils import dumpjson
# df=pd.read_csv('https://archive.materialscloud.org/record/file?filename=magneticmoment_Ef_data.csv&record_id=130')
# df=df[['formula','magnetic_moment']]
# df=df.sample(frac=1)
# df['id']=df.index
# mem=[]
# for i,ii in df.iterrows():
#     info={}
#     info['id']=ii['id']
#     info['magnetic_moment']=ii['magnetic_moment']
#     info['formula']=ii['formula']
#     mem.append(info)
# dumpjson(filename='mag2d_chem.json',data=mem)

# test_size = 0.2
# n_train = int(len(df)*(1-(test_size)))
# n_test = len(df)-n_train

# #for leaderboard
# mem={}
# train={}

# test={}
# train_df = df[:n_train]

# test_df = df[-n_test:]
# for i,ii in train_df.iterrows():
#     train[str(ii['id'])]=ii['magnetic_moment']

# for i,ii in test_df.iterrows():
#     test[str(ii['id'])]=ii['magnetic_moment']
# mem['train']=train
# mem['test']=test
# fname='mag2d_chem_magnetic_moment.json'
# dumpjson(data=mem,filename=fname)

In [8]:
df

,formula,magnetic_moment,id
49,V1Cr1Ge2Te6,5.0780,49
4,Cr1Co1Si2Te6,3.1560,4
212,Ta1Cr1Sn1Ge1Te6,3.0161,212
153,Cr1Sn1Te6P1Au1,2.7459,153
46,Ti1Cr1Si2Te6,2.2800,46
...,...,...,...
57,Y1Cr1Si1Te6P1,2.9670,57
97,Cr1Ge2Te6W1,5.6479,97
60,Ta1Cr1In1Te6Pb1,2.6981,60
152,Cr1Sn1Te6Mo1P1,5.0129,152


,formula,magnetic_moment
0,Cr1Co1Ge1Te6P1,2.8160
1,Cr1Co1Ge2Te6,3.1420
2,Cr1Co1Si1Ge1Te6,3.1490
3,Cr1Co1Si1Te6P1,2.8410
4,Cr1Co1Si2Te6,3.1560
...,...,...
221,Zr1Cr1Si2Te6,2.7489
222,Zr1Cr1Sn1Ge1Te6,2.8139
223,Zr1Cr1Sn1Te6P1,3.0124
224,Zr1Cr1Sn2Te6,2.8007


In [14]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/mag2d_chem_magnetic_moment.json.zip"
js_tag = "mag2d_chem_magnetic_moment.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [15]:
id_data.keys()

dict_keys(['train', 'test'])

In [16]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [17]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [18]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

lgbm = lgb.LGBMRegressor(
    # device="gpu",
    n_estimators=1170,
    learning_rate=0.15,
    num_leaves=273,
)
X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['magnetic_moment'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['magnetic_moment'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
lgbm.fit(X_train,y_train)
pred=lgbm.predict(X_test)
print (mean_absolute_error(y_test,pred))

1.0266912467798093
CPU times: user 1min 11s, sys: 0 ns, total: 1min 11s
Wall time: 4.79 s


In [19]:
import os
f=open('AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv.zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv (deflated 69%)


0